In [35]:
!pip install ydata-profiling

Generate the report

In [38]:
import pandas as pd
from ydata_profiling import ProfileReport

# Load the dataset
wine_data = pd.read_csv('red_wine.csv')

# Generate the profile report
profile = ProfileReport(wine_data, title="Red Wine Dataset Profiling Report", explorative=True)
profile.to_file("red_wine_profiling_report.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Building supervised machine learning with Logistic Regression (LR), Naïve Bayes (NB), Support Vector Machine (SVM), and Decision tree (DT), Random Forest (RF) classifiers, as compared with simple/baseline methods.

In [60]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the dataset
file_path = 'red_wine.csv'
wine_data = pd.read_csv(file_path)

# Encode the target variable
label_encoder = LabelEncoder()
wine_data['type'] = label_encoder.fit_transform(wine_data['type'])

# Separate features and target
X = wine_data.drop(columns='type')
y = wine_data['type']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize models with default parameters
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'SVM (Linear)': SVC(kernel='linear', probability=True),
    'SVM (RBF)': SVC(kernel='rbf', probability=True),
    'Random Forest': RandomForestClassifier()
}

# Dictionary to store results
results = {}

# Train and evaluate each model
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba) if y_proba is not None else 'N/A'
    report = classification_report(y_test, y_pred)

    results[model_name] = {
        'Accuracy': accuracy,
        'AUC': auc,
        'Classification Report': report
    }

# Baseline Model
baseline = DummyClassifier(strategy="most_frequent")
baseline.fit(X_train, y_train)
y_baseline_pred = baseline.predict(X_test)

baseline_accuracy = accuracy_score(y_test, y_baseline_pred)
baseline_auc = 0.5  # Random classifier AUC
baseline_report = classification_report(y_test, y_baseline_pred)

results['Baseline (Most Frequent)'] = {
    'Accuracy': baseline_accuracy,
    'AUC': baseline_auc,
    'Classification Report': baseline_report
}

# Display results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    print(f"Accuracy: {metrics['Accuracy']}")
    print(f"AUC: {metrics['AUC']}")
    print("Classification Report:")
    print(metrics['Classification Report'])
    print("\n" + "="*40 + "\n")


Model: Logistic Regression
Accuracy: 0.7739130434782608
AUC: 0.8838475499092558
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.78      0.78        58
           1       0.77      0.77      0.77        57

    accuracy                           0.77       115
   macro avg       0.77      0.77      0.77       115
weighted avg       0.77      0.77      0.77       115



Model: Naive Bayes
Accuracy: 0.808695652173913
AUC: 0.8901996370235934
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.84      0.82        58
           1       0.83      0.77      0.80        57

    accuracy                           0.81       115
   macro avg       0.81      0.81      0.81       115
weighted avg       0.81      0.81      0.81       115



Model: Decision Tree
Accuracy: 0.808695652173913
AUC: 0.808832425892317
Classification Report:
              precision    recall  f1-score   supp

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fit a model using each of the following methods and report the performance metrics of 10-fold cross-validation using red-wine.csv

In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the dataset
file_path = 'red_wine.csv'
wine_data = pd.read_csv(file_path)

# Encode the target variable
label_encoder = LabelEncoder()
wine_data['type'] = label_encoder.fit_transform(wine_data['type'])

# Separate features and target
X = wine_data.drop(columns='type')
y = wine_data['type']

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Initialize models with default parameters
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'SVM (Linear)': SVC(kernel='linear', probability=True),
    'SVM (RBF)': SVC(kernel='rbf', probability=True),
    'Random Forest': RandomForestClassifier()
}

# Stratified K-Fold Cross Validation
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
cv_results = {}

# Update make_scorer to use only response_method="predict_proba" for AUC scoring
auc_scorer = make_scorer(roc_auc_score, response_method="predict_proba")

# Perform 10-fold cross-validation for each model and calculate accuracy and AUC
for model_name, model in models.items():
    accuracy_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    auc_scores = cross_val_score(model, X, y, cv=kf, scoring=auc_scorer)
    cv_results[model_name] = {
        'Cross-Validation Accuracy': np.mean(accuracy_scores),
        'Cross-Validation AUC': np.mean(auc_scores)
    }

# Baseline Model
baseline = DummyClassifier(strategy="most_frequent")
baseline_accuracy = cross_val_score(baseline, X, y, cv=kf, scoring='accuracy').mean()
baseline_auc = 0.5  # Random classifier AUC

cv_results['Baseline (Most Frequent)'] = {
    'Cross-Validation Accuracy': baseline_accuracy,
    'Cross-Validation AUC': baseline_auc
}

# Display results
cv_results_df = pd.DataFrame(cv_results).T
cv_results_df

,Cross-Validation Accuracy,Cross-Validation AUC
Logistic Regression,0.794858,0.876921
Naive Bayes,0.824773,0.893291
Decision Tree,0.810617,0.807213
SVM (Linear),0.786086,0.877289
SVM (RBF),0.840502,0.924475
Random Forest,0.849153,0.918888
Baseline (Most Frequent),0.528887,0.500000


Plot the ROC curve of the Random Forest classifier from the Python package, and paste a screenshot of your ROC curve here

In [68]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Load the dataset
file_path = 'red_wine.csv'
wine_data = pd.read_csv(file_path)

# Encode the target variable
label_encoder = LabelEncoder()
wine_data['type'] = label_encoder.fit_transform(wine_data['type'])

# Separate features and target
X = wine_data.drop(columns='type')
y = wine_data['type']

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Initialize Random Forest model and Stratified K-Fold
rf_model = RandomForestClassifier()
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Variables to store results for averaging
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

# Perform 10-fold cross-validation
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Train the Random Forest model
    rf_model.fit(X_train, y_train)
    
    # Predict probabilities for the positive class
    y_proba = rf_model.predict_proba(X_test)[:, 1]
    
    # Calculate ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    
    # Interpolate the tpr for consistent mean_fpr points
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0

# Calculate the mean and standard deviation of TPRs
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)

# Plot the mean ROC curve
plt.figure(figsize=(8, 6))
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=0.8)

# Plot the chance line
plt.plot([0, 1], [0, 1], 'k--', lw=2, label="Random Classifier (AUC = 0.5)")

# Fill the standard deviation area
std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='blue', alpha=0.2,
                 label=r'$\pm$ 1 std. dev.')

# Replace plt.show() with plt.savefig() to save the plot as an image file
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve for Random Forest Classifier with 10-Fold Cross-Validation')
plt.legend(loc='best')
plt.savefig("mean_roc_curve_random_forest.png")  # Saves the plot as a PNG file

Using the best model obtained above in Q2 (according to AUC) the best model is SVM (RBF)

In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the dataset
file_path = 'white_wine.csv'
wine_data = pd.read_csv(file_path)

# Encode the target variable
label_encoder = LabelEncoder()
wine_data['type'] = label_encoder.fit_transform(wine_data['type'])

# Separate features and target
X = wine_data.drop(columns='type')
y = wine_data['type']

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Initialize models with default parameters
models = {
    # 'Logistic Regression': LogisticRegression(),
    # 'Naive Bayes': GaussianNB(),
    # 'Decision Tree': DecisionTreeClassifier(),
    # 'SVM (Linear)': SVC(kernel='linear', probability=True),
    'SVM (RBF)': SVC(kernel='rbf', probability=True),
    # 'Random Forest': RandomForestClassifier()
}

# Stratified K-Fold Cross Validation
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
cv_results = {}

# Update make_scorer to use only response_method="predict_proba" for AUC scoring
auc_scorer = make_scorer(roc_auc_score, response_method="predict_proba")

# Perform 10-fold cross-validation for each model and calculate accuracy and AUC
for model_name, model in models.items():
    accuracy_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    auc_scores = cross_val_score(model, X, y, cv=kf, scoring=auc_scorer)
    cv_results[model_name] = {
        'Cross-Validation Accuracy': np.mean(accuracy_scores),
        'Cross-Validation AUC': np.mean(auc_scores)
    }

# Baseline Model
baseline = DummyClassifier(strategy="most_frequent")
baseline_accuracy = cross_val_score(baseline, X, y, cv=kf, scoring='accuracy').mean()
baseline_auc = 0.5  # Random classifier AUC

cv_results['Baseline (Most Frequent)'] = {
    'Cross-Validation Accuracy': baseline_accuracy,
    'Cross-Validation AUC': baseline_auc
}

# Display results
cv_results_df = pd.DataFrame(cv_results).T
cv_results_df

,Cross-Validation Accuracy,Cross-Validation AUC
SVM (RBF),0.873214,0.991667
Baseline (Most Frequent),0.783929,0.500000
